# Digit Recognizer

## Imports, utility functions and global variables

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

In [13]:
IMAGE_WIDTH, IMAGE_HEIGHT = 28, 28
CHANNELS = 1
CLASSES = 10

EPOCHS = 100
BATCH_SIZE=1024

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


## Pre-processing

### Opening the data

In [4]:
df_train = pd.read_csv('data/train.csv')
test_x = pd.read_csv('data/test.csv')

### Dividing each split into X and Y

In [5]:
train_x = df_train.drop(labels = ["label"], axis = 1) 
train_y = df_train['label']

### Data reshaping

In [6]:
train_x = train_x.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)
test_x = test_x.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)

### One-hot encoding

In [7]:
train_y = to_categorical(train_y, num_classes = CLASSES)

## Building the ANN

### Architecture

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='Same', 
                 activation='relu',
                 input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)))
model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3,3),padding='Same', 
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3),padding='Same', 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(CLASSES, activation="softmax"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

### Training and evaluation of the training

In [14]:
model.compile(optimizer=RMSprop(lr=0.0001,
                                rho=0.9,
                                epsilon=1e-08,
                                decay=0.00001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [15]:
history = model.fit(train_x, train_y, verbose=1,validation_split=0.3, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/100
29/29 [==============================] - 73s 3s/step - loss: 2.9699 - accuracy: 0.4458 - val_loss: 0.8169 - val_accuracy: 0.8639
Epoch 2/100
29/29 [==============================] - 73s 3s/step - loss: 0.8528 - accuracy: 0.7129 - val_loss: 0.3554 - val_accuracy: 0.9252
Epoch 3/100
15/29 [==============>...............] - ETA: 32s - loss: 0.5704 - accuracy: 0.8126

KeyboardInterrupt: 